#### 文本相似度检索

In [2]:
from gensim import corpora, models, similarities
from collections import defaultdict
from pprint import pprint  #使打印的格式更齐整
import jieba

#对特定长词进行控制，防止被分错词，影响后续的分析效果
jieba.add_word('微信')      
jieba.add_word('文本挖掘')   
jieba.add_word('增长黑客')   
jieba.add_word('小白')   
jieba.add_word('大数据') 


docs = [
        '数据挖掘实操｜用文本挖掘剖析近5万首《全唐诗》',
        '以虎嗅网4W+文章的文本挖掘为例，展现数据分析的一整套流程',
        '干货｜作为一个合格的“增长黑客”，你还得重视外部数据的分析！',
        '文本挖掘从小白到精通（二）---语料库和词向量空间',
        '文本挖掘从小白到精通（三）---主题模型和文本数据转换',
        '文本挖掘从小白到精通（一）---语料、向量空间和模型的概念',
        '以《大秦帝国之崛起》为例，来谈大数据舆情分析和文本挖掘',
        '文本分类算法集锦，从小白到大牛，附代码注释和训练语料',
        'Social Listening和传统市场调研的关系是怎样的？',
        '【新媒体运营实操】如何做出一个精美的个性化词云'
        '以哈尔滨冰雪大世界旅游的传播效应为例，谈数据新闻可视化的“魅惑”',
        '万字干货｜10款数据分析“工具”，助你成为新媒体运营领域的“增长黑客”',
        '如何用数据分析，搞定新媒体运营的定位和内容初始化？',
        '当数据分析遭遇心理动力学：用户深层次的情感需求浮出水面',
        '揭开微博转发传播的规律：以“人民日报”发布的G20文艺晚会微博为例',
        '数据运营实操 | 如何运用数据分析对某个试运营项目进行“无死角”的复盘？',
        '如何利用微信后台数据优化微信运营',
        '如何利用Social Listening从社会化媒体中“提炼”有价值的信息？',
        '用大数据文本挖掘，来洞察“共享单车”的行业现状及走势',
        '从社交媒体传播和文本挖掘角度解读《欢乐颂2》',
        '不懂数理和编程，如何运用免费的大数据工具获得行业洞察？',
        '写给迷茫的你：如何运用运营思维规划自己的职业发展路径？',
        '如何用聚类分析进行企业公众号的内容优化',
        '傅园慧和她的“洪荒之力”的大数据舆情分析',
        '数据运营|数据分析中，文本分析远比数值型分析重要！（上）'
        ]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\18438\AppData\Local\Temp\jieba.cache
Loading model cost 1.413 seconds.
Prefix dict has been built succesfully.


In [3]:
documents = [' '.join(jieba.lcut(i)) for i in docs]
documents 

['数据挖掘 实操 ｜ 用 文本挖掘 剖析 近 5 万首 《 全唐诗 》',
 '以虎 嗅网 4W + 文章 的 文本挖掘 为例 ， 展现 数据分析 的 一整套 流程',
 '干货 ｜ 作为 一个 合格 的 “ 增长黑客 ” ， 你 还 得 重视 外部 数据 的 分析 ！',
 '文本挖掘 从小 白到 精通 （ 二 ） - - - 语料库 和 词 向量 空间',
 '文本挖掘 从小 白到 精通 （ 三 ） - - - 主题 模型 和 文本 数据 转换',
 '文本挖掘 从小 白到 精通 （ 一 ） - - - 语料 、 向量 空间 和 模型 的 概念',
 '以 《 大秦 帝国 之 崛起 》 为例 ， 来谈 大数据 舆情 分析 和 文本挖掘',
 '文本 分类 算法 集锦 ， 从小 白到 大牛 ， 附 代码 注释 和 训练 语料',
 'Social   Listening 和 传统 市场调研 的 关系 是 怎样 的 ？',
 '【 新 媒体 运营 实操 】 如何 做出 一个 精美 的 个性化 词云以 哈尔滨 冰雪 大 世界 旅游 的 传播效应 为例 ， 谈 数据 新闻 可视化 的 “ 魅惑 ”',
 '万字 干货 ｜ 10 款 数据分析 “ 工具 ” ， 助 你 成为 新 媒体 运营 领域 的 “ 增长黑客 ”',
 '如何 用 数据分析 ， 搞定 新 媒体 运营 的 定位 和 内容 初始化 ？',
 '当 数据分析 遭遇 心理 动力学 ： 用户 深层次 的 情感 需求 浮出 水面',
 '揭开 微博 转发 传播 的 规律 ： 以 “ 人民日报 ” 发布 的 G20 文艺晚会 微博为 例',
 '数据 运营 实操   |   如何 运用 数据分析 对 某个 试运营 项目 进行 “ 无 死角 ” 的 复盘 ？',
 '如何 利用微 信 后台 数据 优化 微信 运营',
 '如何 利用 Social   Listening 从 社会化 媒体 中 “ 提炼 ” 有 价值 的 信息 ？',
 '用 大数据 文本挖掘 ， 来 洞察 “ 共享 单车 ” 的 行业 现状及 走势',
 '从 社交 媒体 传播 和 文本挖掘 角度 解读 《 欢乐颂 2 》',
 '不 懂 数理 和 编程 ， 如何 运用 免费 的 大数据 工具 获得 行业 洞察 ？',
 '写给 迷茫 的 你 ： 

In [4]:
# 去停用词
stop_words = ['：', '。', '?', '|', '《', '》', ':', '“', '”', '，', '、', '+', '｜', '！', '（', '）', '-','【','】','？']
# 移除常用词以及分词 
stoplist = [i.strip() for i in stop_words]
texts = [[word for word in document.lower().split() if word not in stoplist]
for document in documents]

print(texts)

[['数据挖掘', '实操', '用', '文本挖掘', '剖析', '近', '5', '万首', '全唐诗'], ['以虎', '嗅网', '4w', '文章', '的', '文本挖掘', '为例', '展现', '数据分析', '的', '一整套', '流程'], ['干货', '作为', '一个', '合格', '的', '增长黑客', '你', '还', '得', '重视', '外部', '数据', '的', '分析'], ['文本挖掘', '从小', '白到', '精通', '二', '语料库', '和', '词', '向量', '空间'], ['文本挖掘', '从小', '白到', '精通', '三', '主题', '模型', '和', '文本', '数据', '转换'], ['文本挖掘', '从小', '白到', '精通', '一', '语料', '向量', '空间', '和', '模型', '的', '概念'], ['以', '大秦', '帝国', '之', '崛起', '为例', '来谈', '大数据', '舆情', '分析', '和', '文本挖掘'], ['文本', '分类', '算法', '集锦', '从小', '白到', '大牛', '附', '代码', '注释', '和', '训练', '语料'], ['social', 'listening', '和', '传统', '市场调研', '的', '关系', '是', '怎样', '的'], ['新', '媒体', '运营', '实操', '如何', '做出', '一个', '精美', '的', '个性化', '词云以', '哈尔滨', '冰雪', '大', '世界', '旅游', '的', '传播效应', '为例', '谈', '数据', '新闻', '可视化', '的', '魅惑'], ['万字', '干货', '10', '款', '数据分析', '工具', '助', '你', '成为', '新', '媒体', '运营', '领域', '的', '增长黑客'], ['如何', '用', '数据分析', '搞定', '新', '媒体', '运营', '的', '定位', '和', '内容', '初始化'], ['当', '数据分析', '遭遇', '心理', '动力学', '用户', 

In [5]:
# 使用词袋模型提取问文本特征
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [6]:
# 基于构建好的词袋表示语料库，用LSI构建一个10维的向量空间
# 值得注意的一个参数是power_iters，模型中的默认设为2，设置的数值越大，模型的精确度就越高，尤其是在训练语料较少的情况下，但训练语料过多的话，加大该数值会影响模型的运行效率。
lsi = models.LsiModel(
         corpus, 
         id2word=dictionary,
         power_iters=100,
         num_topics=10
         )

现在，笔者设定的检索语句为“文本挖掘在舆情口碑挖掘中的作用很大”（需做分词处理）， 笔者想让结果按相似度的降序进行排列展示，也就是和查询语句相似度越高（相似度数值较大）的展示在上面。

在这里我们只是单纯的考虑文本（词汇）的显式语义相关性

In [7]:
#查询语句为“文本挖掘在舆情口碑挖掘中的作用很大” 
doc = "文本挖掘 在 舆情 口碑 挖掘 中 的 作用 很大 "
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]      #将查询语句转换到LSI向量空间

In [8]:
result = [(docs[i[0]],i[1]) for i in vec_lsi]
pprint(sorted(result,key=lambda x: x[1],reverse=True))

[('以虎嗅网4W+文章的文本挖掘为例，展现数据分析的一整套流程', 0.3619939473795544),
 ('干货｜作为一个合格的“增长黑客”，你还得重视外部数据的分析！', 0.35781006161854967),
 ('文本挖掘从小白到精通（三）---主题模型和文本数据转换', 0.15847743278949084),
 ('文本挖掘从小白到精通（二）---语料库和词向量空间', 0.13785471597129512),
 ('【新媒体运营实操】如何做出一个精美的个性化词云以哈尔滨冰雪大世界旅游的传播效应为例，谈数据新闻可视化的“魅惑”',
  0.05712771001193546),
 ('文本分类算法集锦，从小白到大牛，附代码注释和训练语料', -0.07801000538899819),
 ('以《大秦帝国之崛起》为例，来谈大数据舆情分析和文本挖掘', -0.13559364956572714),
 ('Social Listening和传统市场调研的关系是怎样的？', -0.24000311522407283),
 ('文本挖掘从小白到精通（一）---语料、向量空间和模型的概念', -0.25532803413482563),
 ('数据挖掘实操｜用文本挖掘剖析近5万首《全唐诗》', -0.929597257276769)]


In [9]:
index = similarities.MatrixSimilarity(lsi[corpus])  #将查询语料库转换到LSI向量空间并对其中的每个文档/语句建立索引
#内存友好型接口
#index = similarities.Similarity(output_prefix='Similarity',corpus=lsi[corpus],num_features=500)  #将查询语料库转换到LSI向量空间并对其中的每个文档/语句建立索引

In [10]:
# 实施查询
sims = index[vec_lsi]   
result = [(docs[i[0]],i[1]) for i in enumerate(sims)]            # 对检索语料库进行相似度查询
pprint(sorted(result ,key=lambda x: x[1],reverse=True))          # 每个查询结果的格式是 (语句， 与查询语句的相似度) ，是一个2元元组

[('傅园慧和她的“洪荒之力”的大数据舆情分析', 0.87644345),
 ('以虎嗅网4W+文章的文本挖掘为例，展现数据分析的一整套流程', 0.82108116),
 ('Social Listening和传统市场调研的关系是怎样的？', 0.74482477),
 ('用大数据文本挖掘，来洞察“共享单车”的行业现状及走势', 0.6638191),
 ('以《大秦帝国之崛起》为例，来谈大数据舆情分析和文本挖掘', 0.63695246),
 ('揭开微博转发传播的规律：以“人民日报”发布的G20文艺晚会微博为例', 0.5492105),
 ('干货｜作为一个合格的“增长黑客”，你还得重视外部数据的分析！', 0.51892805),
 ('如何用数据分析，搞定新媒体运营的定位和内容初始化？', 0.5128285),
 ('文本挖掘从小白到精通（一）---语料、向量空间和模型的概念', 0.5031858),
 ('【新媒体运营实操】如何做出一个精美的个性化词云以哈尔滨冰雪大世界旅游的传播效应为例，谈数据新闻可视化的“魅惑”', 0.49039814),
 ('如何用聚类分析进行企业公众号的内容优化', 0.4471987),
 ('从社交媒体传播和文本挖掘角度解读《欢乐颂2》', 0.4446677),
 ('当数据分析遭遇心理动力学：用户深层次的情感需求浮出水面', 0.43081084),
 ('不懂数理和编程，如何运用免费的大数据工具获得行业洞察？', 0.41420436),
 ('如何利用Social Listening从社会化媒体中“提炼”有价值的信息？', 0.39105833),
 ('写给迷茫的你：如何运用运营思维规划自己的职业发展路径？', 0.3840393),
 ('数据挖掘实操｜用文本挖掘剖析近5万首《全唐诗》', 0.32529676),
 ('文本挖掘从小白到精通（三）---主题模型和文本数据转换', 0.31614575),
 ('文本挖掘从小白到精通（二）---语料库和词向量空间', 0.3131905),
 ('数据运营实操 | 如何运用数据分析对某个试运营项目进行“无死角”的复盘？', 0.26436985),
 ('数据运营|数据分析中，文本分析远比数值型分析重要！（上）', 0.25880426),
 ('万字

#### 文本相似性度量

衡量两个文本的相似性有很多种方法，如最直接的利用Hashcode，以及经典的欧氏距离、余弦距离等。  
在比较两个文本集合（主要是基于主题模型的概率分布）的相似程度时，我们需要一些跟欧氏距离、余弦距离不同的相似度/距离度量方案。

In [2]:
from gensim.corpora import Dictionary
from gensim.models import ldamodel
from gensim.matutils import kullback_leibler, jaccard, hellinger, sparse2full
import numpy

texts =  [
  ['苹果','叶子','椭圆形','树上'],           
  ['植物','叶子','绿色','落叶乔木'],           
  ['水果','苹果','红彤彤','味道'],            
  ['苹果','落叶乔木','树上','水果'],           
  ['植物','营养','维生素'],           
  ['营养','维生素','苹果','成分'],            
  ['互联网','电脑','智能手机','高科技'],         
  ['苹果','公司','互联网','品质'],          
  ['乔布斯','苹果','硅谷'],          
  ['电脑','智能手机','苹果','乔布斯'],         
  ['苹果','电脑','品质','生意'],         
  ['电脑','品质','乔布斯'],          
  ['苹果','公司','生意','硅谷']]

dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [5]:
numpy.random.seed(2019) # 设置随机种子以获得每次相同的结果。
model = ldamodel.LdaModel(corpus, id2word=dictionary, iterations=100, num_topics=2)
model.show_topics(num_words=10) #展示主题模型，显示每个主题下的TOP10主题词

[(0,
  '0.170*"苹果" + 0.087*"电脑" + 0.064*"品质" + 0.063*"乔布斯" + 0.053*"智能手机" + 0.052*"水果" + 0.051*"互联网" + 0.049*"树上" + 0.049*"公司" + 0.042*"生意"'),
 (1,
  '0.093*"苹果" + 0.082*"植物" + 0.075*"维生素" + 0.074*"营养" + 0.060*"叶子" + 0.058*"落叶乔木" + 0.051*"成分" + 0.049*"绿色" + 0.048*"硅谷" + 0.045*"乔布斯"')]

基于训练好的主题模型来比较文档之间的相似度

In [6]:
doc1 = ['树上', '叶子', '植物']
doc2 = ['乔布斯', '智能手机', '互联网']
doc3 = [ '营养', '苹果','维生素']


bow1 = model.id2word.doc2bow(doc1)  
bow2 = model.id2word.doc2bow(doc2)   
bow3 = model.id2word.doc2bow(doc3)  

In [7]:
lda_bow1 = model[bow1]
lda_bow2 = model[bow2]
lda_bow3 = model[bow3]

用Hellinger 和 Kullback–Leibler来度量上述语句之间的相似度/距离

In [8]:
print(hellinger(lda_bow1, lda_bow3))
print(hellinger(lda_bow2, lda_bow3))
print(hellinger(lda_bow1, lda_bow2))

0.046444595428538994
0.505161552997494
0.4615085597971655


In [9]:
print(kullback_leibler(lda_bow1, lda_bow3))
print(kullback_leibler(lda_bow2, lda_bow3))
print(kullback_leibler(lda_bow1, lda_bow2))

0.008893967
1.0357668
0.9507062


利用Jaccard来度量文档之间的相似度

两个集合A和B的交集元素在A、B的并集中所占的比例，称为两个集合的杰卡德相似系数，用符号J(A,B)表示。  
它用于比较有限样本集之间的相似性与差异性，Jaccard系数值越大，样本相似度越低

它的典型应用场景有：1 比较文本相似度，用于文本查重与去重 2 计算对象间距离，用于数据聚类等

In [11]:
jaccard(bow1, bow2)

1.0

In [12]:
jaccard(['苹果','大树','营养'], ['苹果','大树','营养'])

0.0

#### 主题分布的距离度量（Distance Metrics for Topic Distributions）

假若，我们想知道这两个主题在语句距离上是多么的相近，该如何做

 度量两个主题间的相似度

In [13]:
topic_公司, topic_水果 = model.show_topics() 

#一些预处理，以使距离度量以可接受的数据形式来获得主题
def make_topics_bow(topic):
# 获取由model.show_topics（）返回的字符串，分割字符串以便分别获取主题和概率    
    topic = topic.split('+')#用于存储主题bows的list    
    topic_bow = []
    for word in topic:#分隔概率和词汇       
        prob, word = word.split('*')#去掉空格       
        word = word.replace(" ","")       
        word = word.replace('"','')#词汇表示转换
        #print(word)       
        word = model.id2word.doc2bow([word])[0][0]       
        topic_bow.append((word, float(prob)))
    return topic_bow

抽取“公司”、“水果”这两个主题的词汇分布，并展示“水果”这一主题的词汇分布情况：

In [14]:
公司_distribution = make_topics_bow(topic_公司[1])
水果_distribution = make_topics_bow(topic_水果[1])

In [15]:
# 以主题词汇加权表示的“水果”主题
水果_distribution

[(3, 0.093),
 (4, 0.082),
 (10, 0.075),
 (11, 0.074),
 (0, 0.06),
 (6, 0.058),
 (12, 0.051),
 (5, 0.049),
 (20, 0.048),
 (19, 0.045)]

In [16]:
hellinger(公司_distribution, 水果_distribution) # 返回值约为0.69，这意味着这2个主题在词汇分布方面存在一定差距
                                               # 深层次的原因是 ---这两个主题所反映的语义存在差异。

0.6917495641343796